[Reference](https://towardsdatascience.com/skorch-pytorch-models-trained-with-a-scikit-learn-wrapper-62b9a154623e)

![SKORCH](https://miro.medium.com/max/1400/1*E4GZb4qpcppgTM6BYN_X_Q.jpeg)


# PyTorch Model

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuralNet(nn.Module):
    def __init__(self, num_units=10, dropout=0.1):
        super(NeuralNet, self).__init__()
        self.num_units = num_units
        self.linear_1 = nn.Linear(13, num_units)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(num_units, 10)
        self.linear_3 = nn.Linear(10, 3)

    def forward(self, x):

        x = self.linear_1(x)
        x = F.relu(x)
        x = self.linear_2(x)
        x = F.relu(x)
        x = self.linear_3(x)
        x = F.softmax(x, dim=-1)

        return x

# Training with SKORCH

## 1. Basic Training

In [4]:
# Import PyTorch model
from src import NeuralNet
# Import SKORCH NN classifier
from skorch import NeuralNetClassifier

# The Neural Net is initialized with fixed hyperparameters
nn = NeuralNetClassifier(NeuralNet, max_epochs=10, lr=0.01, batch_size=12, optimizer=optim.RMSprop)
# Training
nn.fit(self.x, self.y)

## 2. Pipeline: Scaler + Training

In [5]:
# Import PyTorch model
from src import NeuralNet

# Import Pipeline and Scaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Import SKORCH NN classifier
from skorch import NeuralNetClassifier

# The Neural Net is initialized with fixed hyperparameters
nn = NeuralNetClassifier(NeuralNet, max_epochs=10, lr=0.01, batch_size=12, optimizer=optim.RMSprop)
# The pipeline instatiated, it wraps scaling and training phase
pipeline = Pipeline([('scale', StandardScaler()), ('nn', nn)])
# Pipeline execution
pipeline.fit(self.x, self.y)

## 3. Pipeline: Scaler + Training + Callbacks

In [6]:
# PyTorch optimizer
from torch import optim

# Pipeline and Standar scaler import
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Import the epoch scoring callback
from skorch.callbacks import EpochScoring
from skorch import NeuralNetClassifier

from src import NeuralNet

# The EpochScoring from callbacks is initialized
balanced_accuracy = EpochScoring(scoring='balanced_accuracy', lower_is_better=False)
accuracy = EpochScoring(scoring='accuracy', lower_is_better=False)

# The Neural Net is initialized with fixed hyperparameters
nn = NeuralNetClassifier(NeuralNet, max_epochs=10, lr=0.01, batch_size=12, optimizer=optim.RMSprop, callbacks=[balanced_accuracy, accuracy])
# The pipeline instatiated, it wraps scaling and training phase
pipeline = Pipeline([('scale', StandardScaler()), ('nn', nn)])
# Pipeline execution
pipeline.fit(self.x, self.y)

## 4. GridSearch: Pipeline + Scaler + Training + Callbacks

In [7]:
# PyTorch optimizer
from torch import optim

# Pipeline
from sklearn.pipeline import Pipeline
# Scaler
from sklearn.preprocessing import StandardScaler
# Grid Search
from sklearn.model_selection import GridSearchCV

# Scoring with callbacks
from skorch.callbacks import EpochScoring
# Neural Net wrapper
from skorch import NeuralNetClassifier
# PyTorch model
from src import NeuralNet

# The Neural Net is instantiated, none hyperparameter is provided
nn = NeuralNetClassifier(NeuralNet, verbose=0, train_split=False)
# The pipeline is instantiated, it wraps scaling and training phase
pipeline = Pipeline([('scale', StandardScaler()), ('nn', nn)])

# The parameters for the grid search are defined
# It must be used the prefix "nn__" when setting hyperparamters for the training phase
# It must be used the prefix "nn__module__" when setting hyperparameters for the Neural Net
params = {
	'nn__max_epochs':[10, 20],
	'nn__lr': [0.1, 0.01],
	'nn__module__num_units': [5, 10],
	'nn__module__dropout': [0.1, 0.5],
	'nn__optimizer': [optim.Adam, optim.SGD, optim.RMSprop]}

# The grid search module is instantiated
gs = GridSearchCV(pipeline, params, refit=False, cv=3, scoring='balanced_accuracy', verbose=1)
# Initialize grid search
gs.fit(self.x, self.y)